In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3 as sql
import re
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt


%matplotlib inline

In [2]:
Police_calls_1 = pd.read_excel("../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx",1)
Police_calls_2 = pd.read_excel("../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx",2)
Police_calls_3 = pd.read_excel("../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx",3)

In [3]:
Police_calls_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48020 entries, 0 to 48019
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Month          48020 non-null  object
 1   Precinct       48020 non-null  object
 2   Date           47926 non-null  object
 3   Created        47926 non-null  object
 4   Address        47926 non-null  object
 5   Incident Type  47926 non-null  object
 6   Priority       47926 non-null  object
dtypes: object(7)
memory usage: 2.6+ MB


In [4]:
Police_Calls=Police_calls_1.dropna()

In [5]:
Police_Calls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47926 entries, 0 to 48019
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Month          47926 non-null  object
 1   Precinct       47926 non-null  object
 2   Date           47926 non-null  object
 3   Created        47926 non-null  object
 4   Address        47926 non-null  object
 5   Incident Type  47926 non-null  object
 6   Priority       47926 non-null  object
dtypes: object(7)
memory usage: 2.9+ MB


In [6]:
Police_calls_2

,Month,Precinct,ASSIST EMS,DISORDERLY SUBJECT,DOMESTIC DISTURBANCE,INTOXICATED PERSON,MENTAL CONDITION,SUICIDAL PERSON,SUSPICIOUS PERSON,TRESPASSER,WELFARE CHECK
0,August,C,203,354,11,7,28,43,18,104,85
1,August,CMD1,0,1,0,0,0,0,0,0,0
2,August,CW,3,8,0,0,1,1,1,3,2
3,August,E,91,187,56,3,13,16,44,33,88
4,August,H,190,310,64,5,28,31,69,68,190
...,...,...,...,...,...,...,...,...,...,...,...
91,March,MT,149,199,31,7,10,21,53,49,140
92,March,N,122,209,35,1,24,26,34,53,119
93,March,S,198,286,59,3,27,29,42,66,236
94,March,W,87,131,11,4,12,14,46,36,103


In [7]:
Police_calls_3

,Month,Precinct,Priority-1,Priority-2,Priority-2L,Priority-2M,Priority-2N,Priority-2T,Priority-3,Priority-3H,Priority-3M,Priority-3Q,Total
0,August,C,2,578,0,251,0,0,21,0.0,1,0,853
1,August,CMD1,0,0,0,1,0,0,0,0.0,0,0,1
2,August,CW,0,2,0,17,0,0,0,0.0,0,0,19
3,August,E,0,391,0,126,0,0,12,0.0,2,0,531
4,August,H,0,729,0,190,0,0,32,0.0,4,0,955
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,March,MT,0,431,0,217,0,0,11,0.0,0,0,659
92,March,N,0,448,0,139,0,0,33,0.0,3,0,623
93,March,S,1,685,0,220,0,0,40,0.0,0,0,946
94,March,W,0,294,0,132,0,0,16,NaN,2,0,444


In [8]:
conn = sql.connect('../data/census.sqlite')

In [9]:
cur = conn.cursor()

In [10]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

In [11]:
available_tables = (cur.fetchall())

In [12]:
print(available_tables)

[('B01001',), ('B03002',), ('B11001',), ('B11002',), ('B15002',), ('B19001',), ('B19013',), ('B19301',), ('B25024',), ('B25075',), ('S1701',)]


In [13]:
cur.execute("SELECT * FROM 'B01001' LIMIT 5;")

In [14]:
B01001 = (cur.fetchall())

In [15]:
type(B01001)

list

In [16]:
B01001

[(0,
  '1400000US47037010103',
  'Census Tract 101.03, Davidson County, Tennessee',
  10103,
  37,
  47,
  'B01001_001E',
  2533,
  'Estimate!!Total:'),
 (1,
  '1400000US47037010104',
  'Census Tract 101.04, Davidson County, Tennessee',
  10104,
  37,
  47,
  'B01001_001E',
  2955,
  'Estimate!!Total:'),
 (2,
  '1400000US47037010105',
  'Census Tract 101.05, Davidson County, Tennessee',
  10105,
  37,
  47,
  'B01001_001E',
  4435,
  'Estimate!!Total:'),
 (3,
  '1400000US47037010106',
  'Census Tract 101.06, Davidson County, Tennessee',
  10106,
  37,
  47,
  'B01001_001E',
  3397,
  'Estimate!!Total:'),
 (4,
  '1400000US47037010201',
  'Census Tract 102.01, Davidson County, Tennessee',
  10201,
  37,
  47,
  'B01001_001E',
  4157,
  'Estimate!!Total:')]

In [26]:
B01001_df = pd.read_sql("SELECT * FROM B01001;", conn)

In [27]:
type(B01001_df)

pandas.core.frame.DataFrame

In [28]:
B01001_df.head()

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B01001_001E,2533,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B01001_001E,2955,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B01001_001E,4435,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B01001_001E,3397,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B01001_001E,4157,Estimate!!Total:


In [29]:
cur.execute("SELECT * FROM 'B03002' LIMIT 5;")
B03002 = (cur.fetchall())
B03002_df = pd.read_sql("SELECT * FROM B03002;", conn)
B03002_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B03002_001E,2533,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B03002_001E,2955,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B03002_001E,4435,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B03002_001E,3397,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B03002_001E,4157,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
3376,3376,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B03002_021E,0,Estimate!!Total:!!Hispanic or Latino:!!Two or ...
3377,3377,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B03002_021E,59,Estimate!!Total:!!Hispanic or Latino:!!Two or ...
3378,3378,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B03002_021E,14,Estimate!!Total:!!Hispanic or Latino:!!Two or ...
3379,3379,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B03002_021E,0,Estimate!!Total:!!Hispanic or Latino:!!Two or ...


In [30]:
cur.execute("SELECT * FROM 'B11001' LIMIT 5;")
B11001 = (cur.fetchall())
B11001_df = pd.read_sql("SELECT * FROM B11001;", conn)
B11001_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B11001_001E,990,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B11001_001E,1221,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B11001_001E,1860,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B11001_001E,1152,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B11001_001E,1731,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
1444,1444,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B11001_009E,517,Estimate!!Total:!!Nonfamily households:!!House...
1445,1445,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B11001_009E,807,Estimate!!Total:!!Nonfamily households:!!House...
1446,1446,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B11001_009E,265,Estimate!!Total:!!Nonfamily households:!!House...
1447,1447,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B11001_009E,0,Estimate!!Total:!!Nonfamily households:!!House...


In [31]:
cur.execute("SELECT * FROM 'B11002' LIMIT 5;")
B11002 = (cur.fetchall())
B11002_df = pd.read_sql("SELECT * FROM B11002;", conn)
B11002_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B11002_001E,2533,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B11002_001E,2955,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B11002_001E,4430,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B11002_001E,3260,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B11002_001E,4157,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
1927,1927,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B11002_012E,2936,Estimate!!Total:!!In nonfamily households
1928,1928,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B11002_012E,5088,Estimate!!Total:!!In nonfamily households
1929,1929,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B11002_012E,1196,Estimate!!Total:!!In nonfamily households
1930,1930,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B11002_012E,0,Estimate!!Total:!!In nonfamily households


In [32]:
cur.execute("SELECT * FROM 'B15002' LIMIT 5;")
B15002 = (cur.fetchall())
B15002_df = pd.read_sql("SELECT * FROM B15002;", conn)
B15002_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B15002_001E,1872,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B15002_001E,2349,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B15002_001E,3621,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B15002_001E,2301,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B15002_001E,3163,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
5630,5630,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B15002_035E,157,Estimate!!Total:!!Female:!!Doctorate degree
5631,5631,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B15002_035E,58,Estimate!!Total:!!Female:!!Doctorate degree
5632,5632,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B15002_035E,5,Estimate!!Total:!!Female:!!Doctorate degree
5633,5633,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B15002_035E,0,Estimate!!Total:!!Female:!!Doctorate degree


In [33]:
cur.execute("SELECT * FROM 'B19001' LIMIT 5;")
B19001 = (cur.fetchall())
B19001_df = pd.read_sql("SELECT * FROM B19001;", conn)
B19001_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B19001_001E,990,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B19001_001E,1221,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B19001_001E,1860,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B19001_001E,1152,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B19001_001E,1731,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
2732,2732,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B19001_017E,311,"Estimate!!Total:!!$200,000 or more"
2733,2733,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B19001_017E,654,"Estimate!!Total:!!$200,000 or more"
2734,2734,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B19001_017E,99,"Estimate!!Total:!!$200,000 or more"
2735,2735,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B19001_017E,0,"Estimate!!Total:!!$200,000 or more"


In [34]:
cur.execute("SELECT * FROM 'B19013' LIMIT 5;")
B19013 = (cur.fetchall())
B19013_df = pd.read_sql("SELECT * FROM B19013;", conn)
B19013_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B19013_001E,70341,Estimate!!Median household income in the past ...
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B19013_001E,64844,Estimate!!Median household income in the past ...
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B19013_001E,64688,Estimate!!Median household income in the past ...
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B19013_001E,63864,Estimate!!Median household income in the past ...
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B19013_001E,62829,Estimate!!Median household income in the past ...
...,...,...,...,...,...,...,...,...,...
156,156,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B19013_001E,81286,Estimate!!Median household income in the past ...
157,157,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B19013_001E,75446,Estimate!!Median household income in the past ...
158,158,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B19013_001E,59072,Estimate!!Median household income in the past ...
159,159,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B19013_001E,-666666666,Estimate!!Median household income in the past ...


In [35]:
cur.execute("SELECT * FROM 'B19301' LIMIT 5;")
B19301 = (cur.fetchall())
B19301_df = pd.read_sql("SELECT * FROM B19301;", conn)
B19301_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B19301_001E,33867,Estimate!!Per capita income in the past 12 mon...
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B19301_001E,38506,Estimate!!Per capita income in the past 12 mon...
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B19301_001E,35122,Estimate!!Per capita income in the past 12 mon...
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B19301_001E,30843,Estimate!!Per capita income in the past 12 mon...
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B19301_001E,34567,Estimate!!Per capita income in the past 12 mon...
...,...,...,...,...,...,...,...,...,...
156,156,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B19301_001E,58644,Estimate!!Per capita income in the past 12 mon...
157,157,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B19301_001E,66717,Estimate!!Per capita income in the past 12 mon...
158,158,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B19301_001E,30948,Estimate!!Per capita income in the past 12 mon...
159,159,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B19301_001E,-666666666,Estimate!!Per capita income in the past 12 mon...


In [36]:
cur.execute("SELECT * FROM 'B25024' LIMIT 5;")
B25024 = (cur.fetchall())
B25024_df = pd.read_sql("SELECT * FROM B25024;", conn)
B25024_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B25024_001E,1022,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B25024_001E,1382,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B25024_001E,1958,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B25024_001E,1186,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B25024_001E,1826,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
1766,1766,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B25024_011E,0,"Estimate!!Total:!!Boat, RV, van, etc."
1767,1767,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B25024_011E,0,"Estimate!!Total:!!Boat, RV, van, etc."
1768,1768,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B25024_011E,0,"Estimate!!Total:!!Boat, RV, van, etc."
1769,1769,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B25024_011E,0,"Estimate!!Total:!!Boat, RV, van, etc."


In [37]:
cur.execute("SELECT * FROM 'B25075' LIMIT 5;")
B25075 = (cur.fetchall())
B25075_df = pd.read_sql("SELECT * FROM B25075;", conn)
B25075_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037010103,"Census Tract 101.03, Davidson County, Tennessee",10103,37,47,B25075_001E,873,Estimate!!Total:
1,1,1400000US47037010104,"Census Tract 101.04, Davidson County, Tennessee",10104,37,47,B25075_001E,975,Estimate!!Total:
2,2,1400000US47037010105,"Census Tract 101.05, Davidson County, Tennessee",10105,37,47,B25075_001E,1626,Estimate!!Total:
3,3,1400000US47037010106,"Census Tract 101.06, Davidson County, Tennessee",10106,37,47,B25075_001E,885,Estimate!!Total:
4,4,1400000US47037010201,"Census Tract 102.01, Davidson County, Tennessee",10201,37,47,B25075_001E,1356,Estimate!!Total:
...,...,...,...,...,...,...,...,...,...
4342,4342,1400000US47037019400,"Census Tract 194, Davidson County, Tennessee",19400,37,47,B25075_027E,0,"Estimate!!Total:!!$2,000,000 or more"
4343,4343,1400000US47037019500,"Census Tract 195, Davidson County, Tennessee",19500,37,47,B25075_027E,0,"Estimate!!Total:!!$2,000,000 or more"
4344,4344,1400000US47037019600,"Census Tract 196, Davidson County, Tennessee",19600,37,47,B25075_027E,0,"Estimate!!Total:!!$2,000,000 or more"
4345,4345,1400000US47037980100,"Census Tract 9801, Davidson County, Tennessee",980100,37,47,B25075_027E,0,"Estimate!!Total:!!$2,000,000 or more"


In [38]:
cur.execute("SELECT * FROM 'S1701' LIMIT 5;")
S1701 = (cur.fetchall())
S1701_df = pd.read_sql("SELECT * FROM S1701;", conn)
S1701_df

,index,GEO_ID,NAME,tract,county,state,variable,value,description
0,0,1400000US47037015631,"Census Tract 156.31, Davidson County, Tennessee",15631,37,47,S1701_C01_001E,12144.0,Estimate!!Total!!Population for whom poverty s...
1,1,1400000US47037015804,"Census Tract 158.04, Davidson County, Tennessee",15804,37,47,S1701_C01_001E,4093.0,Estimate!!Total!!Population for whom poverty s...
2,2,1400000US47037017701,"Census Tract 177.01, Davidson County, Tennessee",17701,37,47,S1701_C01_001E,2466.0,Estimate!!Total!!Population for whom poverty s...
3,3,1400000US47037017702,"Census Tract 177.02, Davidson County, Tennessee",17702,37,47,S1701_C01_001E,3511.0,Estimate!!Total!!Population for whom poverty s...
4,4,1400000US47037018301,"Census Tract 183.01, Davidson County, Tennessee",18301,37,47,S1701_C01_001E,8222.0,Estimate!!Total!!Population for whom poverty s...
...,...,...,...,...,...,...,...,...,...
29458,29458,1400000US47037015613,"Census Tract 156.13, Davidson County, Tennessee",15613,37,47,S1701_C03_061E,39.2,Estimate!!Percent below poverty level!!UNRELAT...
29459,29459,1400000US47037015620,"Census Tract 156.20, Davidson County, Tennessee",15620,37,47,S1701_C03_061E,58.5,Estimate!!Percent below poverty level!!UNRELAT...
29460,29460,1400000US47037016100,"Census Tract 161, Davidson County, Tennessee",16100,37,47,S1701_C03_061E,92.2,Estimate!!Percent below poverty level!!UNRELAT...
29461,29461,1400000US47037011400,"Census Tract 114, Davidson County, Tennessee",11400,37,47,S1701_C03_061E,36.0,Estimate!!Percent below poverty level!!UNRELAT...
